seperate location terms contain numbers, store them as panda dataframe

In [29]:
!pip install openpyxl

In [32]:
from os import listdir
from os.path import isfile, join
import re
import requests
import json
import pandas as pd

df = pd.read_csv('deepseek_samples.csv', encoding="UTF-8")
#remove duplication
df['doc_num'] = df['file_number'].str[:-2]
df = df.drop_duplicates(subset=['doc_num', 'location'])
sorted_df =df.sort_values(by=['file_number']).reset_index(drop = True)
sorted_df

,Unnamed: 0,file_number,location,length,doc_num
0,0.0,22_137_1,Finanzbehörde Hamburg,21.0,22_137
1,2.0,22_1695_2,DRS. 21,7.0,22_1695
2,8.0,22_2088_1,Flurstraße 15,14.0,22_2088
3,12.0,22_2088_1,Meerweinstraße 26,18.0,22_2088
4,9.0,22_2088_1,Vorhornweg 2,13.0,22_2088
...,...,...,...,...,...
754,847.0,22_8489_2,Paloma-Viertel,15.0,22_8489
755,844.0,22_8489_2,Bornplatzsynagoge,18.0,22_8489
756,845.0,22_8489_2,Oberhafen,10.0,22_8489
757,851.0,22_8489_2,Alsterdorfer Markt,19.0,22_8489


In [33]:
df

,Unnamed: 0,file_number,location,length,doc_num
0,261.0,22_3490_2,- Vu Harburg – östlicher Binnenhafen,36.0,22_3490
1,690.0,22_6149_1,"Denickestraße, Harburg",36.0,22_6149
2,696.0,22_6265_1,"Stellinger Schweiz mit Amsinckpark,",35.0,22_6265
3,485.0,22_5428_1,Zeughausstraße 42,35.0,22_5428
4,753.0,22_7611_1,Stadtteilschule Fischbek-Falkenberg,35.0,22_7611
...,...,...,...,...,...
815,NaN,22_5213_2,Barmode-Süd,NaN,22_5213
817,NaN,22_5213_2,Lagerbrözen,NaN,22_5213
825,NaN,22_5213_2,Üthermord,NaN,22_5213
826,NaN,22_5213_2,Orbäderl,NaN,22_5213


In [34]:
numbers_rows = []
location_rows = []
for _, row in sorted_df.iterrows():
     # Regular expression pattern to check if numbers in a string
     pattern = r"\d"
     # Search for the pattern in the text
     if re.search(pattern, row['location']):
        numbers_rows.append({'location': row['location'].strip(), 'doc_num': row['file_number'][:-2]})
     else:
        location_rows.append({'location': row['location'].strip(), 'doc_num': row['file_number'][:-2]})

num_df = pd.DataFrame(numbers_rows)
loc_df = pd.DataFrame(location_rows)

In [35]:
num_df

,location,doc_num
0,DRS. 21,22_1695
1,Flurstraße 15,22_2088
2,Meerweinstraße 26,22_2088
3,Vorhornweg 2,22_2088
4,Mümmelmannsberg 75,22_2088
...,...,...
241,"Schmiedekoppel, 29, Hamburg",22_6994
242,"Eichenhöhe, 9, Eißendorf",22_6994
243,"Schmiedekoppel, 30, Hamburg",22_6994
244,Neumarkt 40,22_7108


In [36]:
loc_df

,location,doc_num
0,Finanzbehörde Hamburg,22_137
1,Schule Tel Emannstraße,22_2088
2,Stadtteilschule in Bergedorf,22_2088
3,Wolfgang-Borchert-Schule,22_2088
4,Elbinselquartierschule,22_2088
...,...,...
508,Paloma-Viertel,22_8489
509,Bornplatzsynagoge,22_8489
510,Oberhafen,22_8489
511,Alsterdorfer Markt,22_8489


In [37]:
#load the official LGV address table as reference
address_df = pd.read_csv('hamburg_address.csv')
address_df.head(100)

,fid,Thoroughfare,street_name,num_zusatz,cX,cY,stadtteil
0,1,H0600,Halenreie,4,10.161927,53.650829,Volksdorf
1,2,H0600,Halenreie,38,10.162457,53.653394,Volksdorf
2,3,H0600,Halenreie,36,10.162814,53.653190,Volksdorf
3,4,H0600,Halenreie,34,10.161265,53.653007,Volksdorf
4,5,H0600,Halenreie,21,10.160180,53.652818,Volksdorf
...,...,...,...,...,...,...,...
95,96,H0590,Haldesdorfer Straße,27,10.068794,53.597424,Bramfeld
96,97,H0590,Haldesdorfer Straße,26,10.068982,53.596997,Bramfeld
97,98,H0590,Haldesdorfer Straße,25,10.068330,53.597527,Bramfeld
98,99,H0590,Haldesdorfer Straße,24c,10.068267,53.597101,Bramfeld


In [38]:
# merge LGV house number with street name
address_df['combine'] = address_df['street_name'].str.lower() +  ' ' + address_df['num_zusatz']
address_df

,fid,Thoroughfare,street_name,num_zusatz,cX,cY,stadtteil,combine
0,1,H0600,Halenreie,4,10.161927,53.650829,Volksdorf,halenreie 4
1,2,H0600,Halenreie,38,10.162457,53.653394,Volksdorf,halenreie 38
2,3,H0600,Halenreie,36,10.162814,53.653190,Volksdorf,halenreie 36
3,4,H0600,Halenreie,34,10.161265,53.653007,Volksdorf,halenreie 34
4,5,H0600,Halenreie,21,10.160180,53.652818,Volksdorf,halenreie 21
...,...,...,...,...,...,...,...,...
296958,296959,T0260,Klgv. 310 Tarpenhöh,4,9.966001,53.615489,Niendorf,klgv. 310 tarpenhöh 4
296959,296960,T0140,Tannenhof,60b,10.093081,53.686626,Lemsahl-Mellingstedt,tannenhof 60b
296960,296961,T0140,Tannenhof,68,10.090965,53.686579,Lemsahl-Mellingstedt,tannenhof 68
296961,296962,T0140,Tannenhof,71,10.090558,53.686282,Lemsahl-Mellingstedt,tannenhof 71


In [39]:
# small letter geocoded location with house numbers, remove space
num_df['combine'] = num_df['location'].str.lower()
num_df

,location,doc_num,combine
0,DRS. 21,22_1695,drs. 21
1,Flurstraße 15,22_2088,flurstraße 15
2,Meerweinstraße 26,22_2088,meerweinstraße 26
3,Vorhornweg 2,22_2088,vorhornweg 2
4,Mümmelmannsberg 75,22_2088,mümmelmannsberg 75
...,...,...,...
241,"Schmiedekoppel, 29, Hamburg",22_6994,"schmiedekoppel, 29, hamburg"
242,"Eichenhöhe, 9, Eißendorf",22_6994,"eichenhöhe, 9, eißendorf"
243,"Schmiedekoppel, 30, Hamburg",22_6994,"schmiedekoppel, 30, hamburg"
244,Neumarkt 40,22_7108,neumarkt 40


In [49]:
import numpy as np
# compare geocoded house number table with LGV data, return coordinates
# common_array = set(num_df['check'].values).intersection(set(address_df['combine']))
common_df = pd.merge(num_df, address_df, on=['combine'], how='inner')
common_df

,location,doc_num,combine,fid,Thoroughfare,street_name,num_zusatz,cX,cY,stadtteil
0,Flurstraße 15,22_2088,flurstraße 15,283707,F1670,Flurstraße,15,9.870252,53.590755,Lurup
1,Meerweinstraße 26,22_2088,meerweinstraße 26,123389,M1100,Meerweinstraße,26,10.026975,53.586985,Winterhude
2,Vorhornweg 2,22_2088,vorhornweg 2,31045,V1140,Vorhornweg,2,9.883608,53.590261,Lurup
3,Mümmelmannsberg 75,22_2088,mümmelmannsberg 75,116462,M3310,Mümmelmannsberg,75,10.148621,53.529890,Billstedt
4,Tangstedter Landstraße 300,22_2088,tangstedter landstraße 300,43965,T0080,Tangstedter Landstraße,300,10.028731,53.670945,Langenhorn
...,...,...,...,...,...,...,...,...,...,...
68,Brödermannsweg 17,22_5213,brödermannsweg 17,232591,B6120,Brödermannsweg,17,9.978596,53.605886,Groß Borstel
69,Gustav-Falke-Straße 19,22_5213,gustav-falke-straße 19,5517,G3420,Gustav-Falke-Straße,19,9.973422,53.573109,Eimsbüttel
70,Kanalplatz 15,22_5387,kanalplatz 15,196310,K0550,Kanalplatz,15,9.982139,53.467055,Harburg
71,Zeughausstraße 42,22_5428,zeughausstraße 42,12700,Z0170,Zeughausstraße,42,9.974184,53.547752,Neustadt


In [41]:
compare_df = common_df
compare_df.drop(columns=['stadtteil','cX','cY','num_zusatz','street_name','fid','Thoroughfare'],inplace=True) 
compare_df

,location,doc_num,combine
0,Flurstraße 15,22_2088,flurstraße 15
1,Meerweinstraße 26,22_2088,meerweinstraße 26
2,Vorhornweg 2,22_2088,vorhornweg 2
3,Mümmelmannsberg 75,22_2088,mümmelmannsberg 75
4,Tangstedter Landstraße 300,22_2088,tangstedter landstraße 300
...,...,...,...
68,Brödermannsweg 17,22_5213,brödermannsweg 17
69,Gustav-Falke-Straße 19,22_5213,gustav-falke-straße 19
70,Kanalplatz 15,22_5387,kanalplatz 15
71,Zeughausstraße 42,22_5428,zeughausstraße 42


In [42]:
no_match = set(num_df['combine']) - set (compare_df['combine'])

no_match_df = pd.DataFrame(columns=num_df.columns)
no_match_df

,location,doc_num,combine


In [43]:
no_match

{'- blohmstr. 22',
 '- borgfelder str. 16',
 '- brennerstr. 56',
 '- entry 35: "farnstraße',
 '- grade: a 13 h.d.',
 '- grade: e 9b',
 '- mitte altona, 2. bauabschnitt',
 '- osterbrookhöfe, 1. bauabschnitt',
 '- osterbrookhöfe, 2. bauabschnitt',
 '- region 1: baakenhafen 4',
 '- umweltwache f32',
 '21149 hamburg',
 '50 the denkmalschutzamt',
 'ahrenshooper straße 1-3',
 'alter hichtweg 200',
 'alternierter straße 38',
 'am feld 12',
 'am luisengymnasium, 0, bergedorf',
 'am überwinterungshafen, 143,',
 'amt f 22, westphalensweg 13',
 'amt m, hammer straße 30-34',
 'an der seebek, 3, bergedorf',
 'anckelmannstraße, 104,',
 'angerstraße, 339,',
 'angerstraße, 48,',
 'baakenhafen 4',
 'baererstraße, 0, harburg',
 'bahrsfmfeller straße 260',
 'barmbek-süd): 160 apartments',
 'barmotherapie 2',
 'baurstraße 20-22',
 'bei der paul-gerhardt-kirche 1-3',
 'berlin saimen 1',
 'billstraße 82-84',
 'borgsweg 176',
 'bovestraße, 6, wandsbek eilbek',
 'brandenbaum park 6',
 'brassenhof 3 (27)',
 'b

In [44]:
no_match_df = num_df[num_df['combine'].isin(no_match)].sort_values(by='doc_num').reset_index(drop=True)
no_match_df

,location,doc_num,combine
0,DRS. 21,22_1695,drs. 21
1,Struenseestraße 20-32,22_2088,struenseestraße 20-32
2,Baakenhafen 4,22_2088,baakenhafen 4
3,Neue Grundschule Eschenweg 1,22_2088,neue grundschule eschenweg 1
4,- Region 1: Baakenhafen 4,22_2088,- region 1: baakenhafen 4
...,...,...,...
168,"Schmiedekoppel, 29, Hamburg",22_6994,"schmiedekoppel, 29, hamburg"
169,"Eichenhöhe, 9, Eißendorf",22_6994,"eichenhöhe, 9, eißendorf"
170,"Schmiedekoppel, 30, Hamburg",22_6994,"schmiedekoppel, 30, hamburg"
171,Neumarkt 40,22_7108,neumarkt 40


In [45]:
no_match_df['latitude'] = 0
no_match_df['longitude'] = 0
no_match_df

,location,doc_num,combine,latitude,longitude
0,DRS. 21,22_1695,drs. 21,0,0
1,Struenseestraße 20-32,22_2088,struenseestraße 20-32,0,0
2,Baakenhafen 4,22_2088,baakenhafen 4,0,0
3,Neue Grundschule Eschenweg 1,22_2088,neue grundschule eschenweg 1,0,0
4,- Region 1: Baakenhafen 4,22_2088,- region 1: baakenhafen 4,0,0
...,...,...,...,...,...
168,"Schmiedekoppel, 29, Hamburg",22_6994,"schmiedekoppel, 29, hamburg",0,0
169,"Eichenhöhe, 9, Eißendorf",22_6994,"eichenhöhe, 9, eißendorf",0,0
170,"Schmiedekoppel, 30, Hamburg",22_6994,"schmiedekoppel, 30, hamburg",0,0
171,Neumarkt 40,22_7108,neumarkt 40,0,0


In [25]:
# call maapbox api for location tables that do not contain house numbers.

from time import sleep

def get_lat_lon(address):
    mapbox_token = "pk.eyJ1IjoiYXJpZWw1MDUiLCJhIjoiY20xcDZ0N3FkMDFhMzJucjJ4N29pbmkydyJ9.86xdZMNPXsuFeB-36Zr58Q"  # Replace with your actual Mapbox API key
    url = f"https://api.mapbox.com/search/geocode/v6/forward?q={address}"
    params = {
        'access_token': mapbox_token,
        'limit': 1,
        'language': 'de',  # German language
        'bbox': '9.65067,53.33928,10.39779,53.75625'  # Bounding box for Hamburg area
    }
   
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Raise an exception for bad status codes
            data = response.json()
            
            if data['features']:
                coordinates = data['features'][0]['geometry']['coordinates']
                print(f"Location: {address}, Coordinates: {coordinates}")
                return address, coordinates[1], coordinates[0]
            else:
                print(f"No results found for address: {address}")
                return address, None, None
        except requests.exceptions.RequestException as e:
            print(f"Error: {e} for address: {address}")
            if attempt < max_retries - 1:
                print(f"Retrying... (Attempt {attempt + 2}/{max_retries})")
                sleep(1)  # Wait for 1 second before retrying
            else:
                print(f"Max retries reached for address: {address}")
                return address, None, None

def geocode_dataframe(no_match_df, location_column='location'):
    geocoded_data = []
    for address in no_match_df[location_column]:
        location, lat, lon = get_lat_lon(address)
        geocoded_data.append({'location': location, 'latitude': lat, 'longitude': lon})
        sleep(0.1)  # Add a small delay to avoid hitting rate limits
    
    geocoded_df = pd.DataFrame(geocoded_data)
    return pd.concat([no_match_df, geocoded_df[['latitude', 'longitude']]], axis=1)

# Usage example
geocoded_df = geocode_dataframe(no_match_df)
print(geocoded_df)

Location: DRS. 21, Coordinates: [9.989817, 53.458348]
No results found for address: - Region 1: Baakenhafen 4
Location: Struenseestraße 20-32, Coordinates: [9.94634, 53.547476]
No results found for address: Baakenhafen 4
Location: Neue Grundschule Eschenweg 1, Coordinates: [10.306554, 53.466294]
Location: Exerzierplatz, 0, Harburg, Coordinates: [9.966737, 53.457595]
Location: Papenstraße, 12, Wandsbek Eilbek, Coordinates: [10.038939, 53.564737]
Location: Baererstraße, 0, Harburg, Coordinates: [9.987821, 53.453656]
Location: Hastedtstraße, 0, Harburg, Coordinates: [9.979226, 53.457081]
Location: Bovestraße, 6, Wandsbek Eilbek, Coordinates: [10.080196, 53.572369]
Location: Papenstraße, 6, Wandsbek Eilbek, Coordinates: [10.038617, 53.56468]
Location: Steinickestraße, 0, Harburg, Coordinates: [9.976974, 53.455365]
Location: Am Luisengymnasium, 0, Bergedorf, Coordinates: [10.37225, 53.43078]
Location: Knoopstraße, 0, Harburg, Coordinates: [9.976792, 53.460527]
Location: Schloßmühlendamm, 0,

In [55]:
geocoded_df

,location,doc_num,latitude,longitude
0,Finanzbehörde Hamburg,22_137,53.550341,10.000654
1,Schule Tel Emannstraße,22_2088,NaN,NaN
2,Stadtteilschule in Bergedorf,22_2088,53.434192,10.371495
3,Wolfgang-Borchert-Schule,22_2088,53.622957,9.865630
4,Elbinselquartierschule,22_2088,NaN,NaN
...,...,...,...,...
508,Paloma-Viertel,22_8489,53.463026,9.971797
509,Bornplatzsynagoge,22_8489,NaN,NaN
510,Oberhafen,22_8489,53.545158,10.006878
511,Alsterdorfer Markt,22_8489,53.612342,10.024278


In [50]:
common_df

,location,doc_num,combine,fid,Thoroughfare,street_name,num_zusatz,cX,cY,stadtteil
0,Flurstraße 15,22_2088,flurstraße 15,283707,F1670,Flurstraße,15,9.870252,53.590755,Lurup
1,Meerweinstraße 26,22_2088,meerweinstraße 26,123389,M1100,Meerweinstraße,26,10.026975,53.586985,Winterhude
2,Vorhornweg 2,22_2088,vorhornweg 2,31045,V1140,Vorhornweg,2,9.883608,53.590261,Lurup
3,Mümmelmannsberg 75,22_2088,mümmelmannsberg 75,116462,M3310,Mümmelmannsberg,75,10.148621,53.529890,Billstedt
4,Tangstedter Landstraße 300,22_2088,tangstedter landstraße 300,43965,T0080,Tangstedter Landstraße,300,10.028731,53.670945,Langenhorn
...,...,...,...,...,...,...,...,...,...,...
68,Brödermannsweg 17,22_5213,brödermannsweg 17,232591,B6120,Brödermannsweg,17,9.978596,53.605886,Groß Borstel
69,Gustav-Falke-Straße 19,22_5213,gustav-falke-straße 19,5517,G3420,Gustav-Falke-Straße,19,9.973422,53.573109,Eimsbüttel
70,Kanalplatz 15,22_5387,kanalplatz 15,196310,K0550,Kanalplatz,15,9.982139,53.467055,Harburg
71,Zeughausstraße 42,22_5428,zeughausstraße 42,12700,Z0170,Zeughausstraße,42,9.974184,53.547752,Neustadt


In [51]:
import openpyxl
common_df.to_excel('geocode_deepseek_lgv.xlsx', index=False)
geocoded_df.to_excel('geocode_deepseek_mapbox.xlsx', index=False)

In [52]:
#geocoded addresses without house numbers with mapbox
geocoded_df = geocode_dataframe(loc_df)
print(geocoded_df)

Location: Finanzbehörde Hamburg, Coordinates: [10.000654, 53.550341]
No results found for address: Schule Tel Emannstraße
Location: Stadtteilschule in Bergedorf, Coordinates: [10.371495, 53.434192]
Location: Wolfgang-Borchert-Schule, Coordinates: [9.86563, 53.622957]
No results found for address: Elbinselquartierschule
Location: Heinrich-Helbing-Schule, Coordinates: [10.225871, 53.486153]
Location: Kiel er Straße, Coordinates: [10.303616, 53.524419]
No results found for address: ehemals an der Seebeck)
Location: Grundschule am Park, Coordinates: [10.392525, 53.749922]
Location: Eilbektal, Coordinates: [10.050823, 53.573157]
Location: Schule Sinstorf, Coordinates: [9.97602, 53.42438]
Location: Schule Ballerstaedtweg, Coordinates: [10.033888, 53.611339]
No results found for address: Deutsch-französisches Gymnasium
Location: Bundesstraße, Coordinates: [10.375612, 53.488439]
Location: knotenpunkt am Luisenhof, Coordinates: [10.11162, 53.60683]
Location: Bramfelder Weg, Coordinates: [10.108

In [ ]:
geocoded_df.to_excel('geocode_deepseek_no_numbers.xlsx', index=False)